In [31]:
# prescribed norm: self-quarantine

In [48]:
from mesa import Agent, Model
from mesa.time import RandomActivation, BaseScheduler
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import numpy as np
import math
import random
import csv
from statistics import mean
from pathlib import Path
# import tensorflow as tf
import pandas as pd
from collections import deque
import time
import os
from tqdm import tqdm

In [49]:
### Simulation Parameters ###

file_name = 'hint_prob'

# general
num_train_iterations = 70
num_eval_iterations = 20
num_steps = 2000
num_agents = 100
agent_per_home = 1

startingState = 0.3
vaccine_multiplier = 0.5
home_recovery_multiplier = 2

num_grocery_stores = 1
num_parks = 1

FQ_frames = 5

# locations
HOME = 0
PARK = 1
GROCERY_STORE = 2
HOSPITAL = 3

# emotion
POSITIVE = 1
NEUTRAL = 0
NEGATIVE = -1

vaccine_last = num_steps#10

location_space = [HOME, PARK, GROCERY_STORE, HOSPITAL]

NOT_INFECTED = 0
INFECTED_A = 1
INFECTED_S = 2
CRITICAL = 3
DECEASED = 4

health_space = [NOT_INFECTED, INFECTED_A, INFECTED_S, CRITICAL, DECEASED]

evolve_prob = [0.8, 0.36, 0.01, 0.2, 0]
recover_prob = [0, 0.2, 0.1, 0.05, 0]

#oberve prob
# N_prob = [0.8,0,0.9,1]
# A_prob = [0.5,0,1,1]
# S_prob = [0.3,0,0.9,1]
# C_prob = [0.1,0,0.4,1]
# N_prob = [0.5,0.8,0.9,1]
# A_prob = [0.3,0.7,1,1]
# S_prob = [0.2,0.4,0.9,1]
# C_prob = [0.1,0.2,0.4,1]
N_prob = [0.6,0.8,0.9,1]
A_prob = [0.4,0.6,1,1]
S_prob = [0.2,0.4,0.9,1]
C_prob = [0.1,0.2,0.4,1]

#signal distribution
hard_sanctioning_prob = 0.20 #0.2
message_sanctioning_prob = 0.0#0.6
emotion_sanctioning_prob = 0.38#1
sanction_prob_w_health = [0,0,0.5,0.8]

# message_prob: prob for sanction 20% message(belief reward) 38% 
# emote_prob: prob for sanction 20% emotion 38%
# hint_prob: prob for sanction 20% emotion 38%
# hint_balance: prob for sanction 20% emotion 38%

# sanction_weighted: prob for sanction 16%
# message_weighted: prob for message 32% 
# feeling_weighted: prob for emotion 32%
# hint_weighted: prob for emotion 20%   -> 20 * 0.5 + 20 * 0.3       

# sanction_weighted2: prob for sanction 28%
# message_weighted2: prob for sanction 20% message 16% 
# feeling_weighted2: prob for sanction 20% emotion 16%
# hint_weighted2: prob for sanction 20% emotion 10%   -> 10 * 0.5 + 10 * 0.3
 
# desire/intention
stay_home = 0
go_outdoors = 1
go_shopping = 2

intention_space = [stay_home, go_outdoors, go_shopping]

# actions
MOVE_HOME = 0
MOVE_PARK = 1
MOVE_STORE = 2
MOVE_HOSPITAL = 3

action_space = [MOVE_HOME, MOVE_PARK, MOVE_STORE, MOVE_HOSPITAL]
sanction_space = ['location', 'health']

env_path = ''
figure_path = env_path + 'figures/' + file_name + '/'
logs_path = env_path + 'logs/' + file_name + '/'

lr = 0.001#0.00025
gamma = 0.9
epsilon_greedy = 0.9
epsilon_min = 0.1

agent_type_primitive = 1
agent_type_sanctioning = 2
agent_type_message = 3
agent_type_emote = 4
agent_type_hint = 5

agent_type = agent_type_hint

SELF = 0
OTHERS = 1

sanction_payoff = 1.0
emotion_payoff = 0.5

# tuples of norms
norm = {
            "type": "prohibition",
            "subject": SELF,
            "object": OTHERS,
            "antecedent": {
                "location": [PARK, GROCERY_STORE, HOSPITAL],
                "attribute": "perceived_health",
                "value": [INFECTED_S, CRITICAL]
            }, 
            "consequent": {
                "emotion": -0.3, 
                "message": -0.5,
                "hard_sanction": -1
            }
        }

In [50]:
# The basic Q learning model
class RL(object):
    def __init__(self, actions):
        self.actions = actions
        self.lr = lr
        self.gamma = gamma
        self.epsilon_greedy = epsilon_greedy
        
        self.q_table = pd.DataFrame(columns = self.actions, dtype = np.float64)
    
    def instantialize(self):
        return
    
    def check_state_exist(self, state):
        if state not in self.q_table.index:
            self.q_table.loc[state] = pd.Series(
                [0] * len(self.actions),
                index = self.q_table.columns
                )

    def action_select(self, state, n_iteration, evaluate):
        self.check_state_exist(state)
        if evaluate or (np.random.uniform() > 0.9 * (num_train_iterations - n_iteration) / num_train_iterations):
            actions= self.q_table.loc[state, :]
            action = np.random.choice(actions[actions == np.max(actions)].index)
        else:
            action = np.random.choice(action_space)
        return action
    
    def getQ(self, state, action):
        return self.q_table((state, action), 0.0)

class QLearningTable(RL):
    def __init__(self, actions):
        super(QLearningTable, self).__init__(actions)
    
    def learn(self, s, a, r, s_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        q_target = r + self.gamma * self.q_table.loc[s_, :].max()
        # check NaN
        q_target = 0 if np.isnan(q_target) else q_target
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)

In [51]:
RL_brain = QLearningTable(actions = list(range(len(action_space))))
Sanction_brain = QLearningTable(actions = list(range(len(sanction_space))))

In [52]:
class HumanAgent(Agent):
    """The actor in the simulation."""
    def __init__(self, unique_id, model, evaluate):
        super().__init__(unique_id, model)
        
        self.homeId = unique_id // agent_per_home
        self.health = NOT_INFECTED
        self.perceived_health = NOT_INFECTED
        
        self.x = HOME
        self.y = unique_id // agent_per_home
        
        self.infected_times = 0
        
        self.external_reward = 0
        
        self.FQ_frames = 0
        
        self.last_action = None
        
        self.vaccinated = False
        self.vaccine_last = 0
        self.intention = 0
        self.self_directed_emotion = 0
        self.other_directed_emotion = 0
        
        self.evaluate = evaluate
        self.reward = 0
        
        self.reset_death = False
    
    def initialize(self):
        self.state = self.get_state(0)
    
    def step(self):
        
        if self.health != DECEASED:
            self.act()
        self.updateHealth()
    
    # observation
    def get_state(self, env_emotion = None):
        loc_x, loc_y = self.pos
        return [loc_x, loc_y, self.health, self.intention, self.FQ_frames, self.vaccinated, self.vaccine_last]
    
    # symptom progress
    def updateHealth(self):
        if self.health == DECEASED:
           return

        x,y = self.pos
        
        if self.health in (INFECTED_A, INFECTED_S, CRITICAL):
            p = self.random.uniform(0, 1)
            ev_prob = evolve_prob[self.health] * (vaccine_multiplier if self.vaccinated else 1)
            rec_prob = recover_prob[self.health] * (home_recovery_multiplier if x == HOME else 1)
            if p < ev_prob:
                self.health += 1
            elif p < ev_prob + rec_prob:
                self.health = NOT_INFECTED
            
            if self.health == NOT_INFECTED:
                if p < N_prob[NOT_INFECTED]:
                    self.perceived_health = NOT_INFECTED
                elif p < N_prob[INFECTED_A]:
                    self.perceived_health = INFECTED_A
                elif p < N_prob[INFECTED_S]:
                    self.perceived_health = INFECTED_S
                elif p < N_prob[CRITICAL]:
                    self.perceived_health = CRITICAL
            elif self.health == INFECTED_A:
                if p < A_prob[NOT_INFECTED]:
                    self.perceived_health = NOT_INFECTED
                elif p < A_prob[INFECTED_A]:
                    self.perceived_health = INFECTED_A
                elif p < A_prob[INFECTED_S]:
                    self.perceived_health = INFECTED_S
                elif p < A_prob[CRITICAL]:
                    self.perceived_health = CRITICAL        
            elif self.health == INFECTED_S:
                if p < S_prob[NOT_INFECTED]:
                    self.perceived_health = NOT_INFECTED
                elif p < S_prob[INFECTED_A]:
                    self.perceived_health = INFECTED_A
                elif p < S_prob[INFECTED_S]:
                    self.perceived_health = INFECTED_S
                elif p < S_prob[CRITICAL]:
                    self.perceived_health = CRITICAL
            elif self.health == CRITICAL:
                if p < C_prob[NOT_INFECTED]:
                    self.perceived_health = NOT_INFECTED
                elif p < C_prob[INFECTED_A]:
                    self.perceived_health = INFECTED_A
                elif p < C_prob[INFECTED_S]:
                    self.perceived_health = INFECTED_S
                elif p < C_prob[CRITICAL]:
                    self.perceived_health = CRITICAL
            elif self.health == DECEASED:
                self.model.deceasedCount += 1
                self.perceived_health = DECEASED
                self.reset_death = True
                return
            
            
                    
        if self.FQ_frames > 0:
            self.FQ_frames -= 1
        return
    
    def perceive(self):
        x, y = self.pos
        return self.model.perceive(x, y, self.unique_id)
    
    # infect with a probability when interact
    def infect(self, close_contact):
        p = self.random.uniform(0, 1)
        if self.health == NOT_INFECTED:
            if close_contact:
                self.health += 1
                self.infected_times += 1
            else:
                # disease proceed with different rate
                if p < (evolve_prob[NOT_INFECTED] * vaccine_multiplier) and self.vaccinated:
                    self.health += 1
                    self.infected_times += 1
                elif p < evolve_prob[NOT_INFECTED] and not self.vaccinated:
                    self.health += 1
                    self.infected_times += 1
    
    def act(self):
        x, y = self.pos
        
        # action selection
        self.intention = np.random.choice(intention_space)
        
        if self.FQ_frames > 0:
            self.last_action = MOVE_HOME
            if x != HOME:
                self.model.grid.move_agent(self, (HOME, self.unique_id // agent_per_home))
                self.x = HOME
                self.y = self.unique_id // agent_per_home
        else:
            action = RL_brain.action_select(str(self.state), self.model.iteration, self.evaluate)
            # run action
            if action == MOVE_HOME:
                self.model.grid.move_agent(self, (HOME, self.unique_id // agent_per_home))
                self.x = HOME
                self.y = self.unique_id // agent_per_home
            elif action == MOVE_STORE:
                rd = self.random.randint(0, num_grocery_stores - 1)
                self.model.grid.move_agent(self, (GROCERY_STORE, rd))
                self.x = GROCERY_STORE
                self.y = rd
            elif action == MOVE_PARK:
                rd = self.random.randint(0, num_parks - 1)
                self.model.grid.move_agent(self, (PARK, self.random.randint(0, num_parks - 1)))
                self.x = PARK
                self.y = rd
            elif action == MOVE_HOSPITAL:
                self.model.grid.move_agent(self, (HOSPITAL, 0))
                self.x = HOSPITAL
                self.y = 0
                self.vaccinated = True
                self.vaccine_last = vaccine_last
 
            self.last_action = action

        if agent_type in (agent_type_emote, agent_type_hint):
            self.elicit_emotion(self.intention)
    
    # elicit self-directed emotion
    def elicit_emotion(self, intention):
        x, y = self.pos
        
        # trigger negative emotions while being forced to stay at home
        if x == HOME and self.FQ_frames > 0:
            self.self_directed_emotion = -emotion_payoff
        else:
            emotion = (1 if x == intention else -1)
            # trigger emotions based on norms
            if x != HOME and self.health > 0:
                emotion += -emotion_payoff
            elif x == HOME and self.health > 0:
                emotion += emotion_payoff
    
            if emotion > emotion_payoff: emotion = emotion_payoff
            if emotion < -emotion_payoff: emotion = -emotion_payoff
            self.self_directed_emotion = emotion
        

    def learn(self, force = False):
        x, y = self.pos
        pre_state = self.state
        
        self.reward = 0
        env_emotion = 0
        
        if agent_type in (agent_type_emote, agent_type_hint):
            self.state = self.get_state(0)
        else:
            self.state = self.get_state(0)
        
        if self.health == DECEASED and not self.reset_death:
            if self.x != HOME or self.x != -1:
                self.model.grid.move_agent(self, (HOME, self.unique_id // agent_per_home))
                self.x = HOME
                self.y = self.unique_id // agent_per_home
            self.reward, self.FQ_frames = 0, 0
            self.self_directed_emotion, self.other_directed_emotion = 0, 0
            self.intention, self.last_action, self.x, self.y = -1, -1, -1, -1
            return
        
        reward = -2 if self.health == DECEASED else 0
        
        if self.FQ_frames > 0:
            reward -= 1
        
        reward += 1 if self.intention == x else -1
        
        if agent_type in (agent_type_emote, agent_type_hint):
            reward += (self.self_directed_emotion + self.other_directed_emotion)
        
        reward += self.external_reward
        
        self.reward = reward
        
        self.reset_death = False
        self.external_reward = 0
        
        if self.evaluate:
            return
        
        RL_brain.learn(str(pre_state), self.last_action, reward, str(self.state))        

In [53]:
class SimulationModel(Model):
    """The model runs the simulation."""
    def __init__(self, N, iteration, evaluate):
        self.num_agents = N
        self.random.seed(1)
        self.evaluate = evaluate
        
        self.grid = MultiGrid(5, num_agents // agent_per_home, False)
        self.schedule = RandomActivation(self)
        self.iteration = iteration
        
        self.deceasedCount = 0
        
        self.datacollector = DataCollector(
            model_reporters={
            }
        )
        
        self.reset()
    
    def reset(self):
        if self.schedule.get_agent_count() > 0:
            for agent in self.schedule.agents:
                self.schedule.remove(agent)
        # load agents
        for i in range(self.num_agents):
            a = HumanAgent(i, self, self.evaluate)
            self.schedule.add(a)
            self.grid.place_agent(a, (HOME, 0))
        
        for a in self.random.sample(self.schedule.agents, int(startingState * self.num_agents)):
            a.health = INFECTED_A
            a.infected_times += 1
        
        for agent in self.schedule.agents:
            agent.initialize()
        
        self.deceased_count = 0
        self.violation_count = 0
        self.compliance_count = 0
        self.cumulative_deceased = 0
        self.cumulative_violation = 0
        self.cumulative_compliance = 0
    
    def run_norms(self):
        emotion_saction_set, message_sanction_set, hard_saction_set = set(), set(), set()
        # Home
        for y in range(num_agents // agent_per_home):
            agents = self.grid.get_cell_list_contents([(HOME,y)])
            comply_agents = [agent for agent in agents if agent.health != DECEASED and agent.perceived_health != NOT_INFECTED and agent.FQ_frames == 0 ]
            for agent in comply_agents:
                agent.external_reward -= norm.get("consequent").get("hard_sanction")
                if agent_type in (agent_type_hint, agent_type_emote):
                    agent.other_directed_emotion = emotion_payoff
        # Park
        if PARK in norm.get("antecedent").get("location"):
            for y in range(num_parks):
                agents = self.grid.get_cell_list_contents([(PARK,y)])
                agents = [agent for agent in agents if agent.health != DECEASED ]
                violate_agents = [agent for agent in agents if getattr(agent, norm.get("antecedent").get("attribute")) in norm.get("antecedent").get("value") ]
                
                if len(agents) - len(violate_agents) > 0:
                    for agent in violate_agents:
                        sanctioners = [sanctioner for sanctioner in agents if sanctioner.health == NOT_INFECTED and sanctioner.unique_id != agent.unique_id]    

                        hard_sanction, sanction = False, False
                        p = self.random.uniform(0, 1)
                        if p < sanction_prob_w_health[agent.perceived_health]:
                            p = self.random.uniform(0, 1)
                            if p < hard_sanctioning_prob:
                                agent.FQ_frames = FQ_frames
                                hard_sanction = True
                            
                            elif agent_type == agent_type_message:
                                if p < message_sanctioning_prob:
                                    sanction = True
                                    agent.external_reward += norm.get("consequent").get("message")
                            elif agent_type in (agent_type_hint, agent_type_emote):
                                if p < emotion_sanctioning_prob:
                                    sanction = True
                                    if agent_type == agent_type_hint:
                                        agent.external_reward += norm.get("consequent").get("emotion")
                                    agent.other_directed_emotion = -emotion_payoff

                        if agent_type in (agent_type_hint, agent_type_emote) and sanction:
                            for sanctioner in sanctioners:
                                if sanctioner.unique_id not in emotion_saction_set:
                                    emotion_saction_set.add(sanctioner.unique_id)
                                    sanctioner.other_directed_emotion = 0

        # Grocery
        if GROCERY_STORE in norm.get("antecedent").get("location"):
            for y in range(num_grocery_stores):
                agents = self.grid.get_cell_list_contents([(GROCERY_STORE,y)])
                agents = [agent for agent in agents if agent.health != DECEASED ]
                violate_agents = [agent for agent in agents if getattr(agent, norm.get("antecedent").get("attribute")) in norm.get("antecedent").get("value") ]
                
                if len(agents) - len(violate_agents) > 0:
                    for agent in violate_agents:
                        sanctioners = [sanctioner for sanctioner in agents if sanctioner.health == NOT_INFECTED and sanctioner.unique_id != agent.unique_id]    
                        
                        hard_sanction, sanction = False, False
                        p = self.random.uniform(0, 1)
                        if p < sanction_prob_w_health[agent.perceived_health]:
                            p = self.random.uniform(0, 1)
                            if p < hard_sanctioning_prob:
                                agent.FQ_frames = FQ_frames
                                hard_sanction = True
                            
                            elif agent_type == agent_type_message:
                                if p < message_sanctioning_prob:
                                    sanction = True
                                    agent.external_reward += norm.get("consequent").get("message")
                            elif agent_type in (agent_type_hint, agent_type_emote):
                                if p < emotion_sanctioning_prob:
                                    sanction = True
                                    if agent_type == agent_type_hint:
                                        agent.external_reward += norm.get("consequent").get("emotion")
                                    agent.other_directed_emotion = -emotion_payoff

                        if agent_type in (agent_type_hint, agent_type_emote) and sanction:
                            for sanctioner in sanctioners:
                                if sanctioner.unique_id not in emotion_saction_set:
                                    emotion_saction_set.add(sanctioner.unique_id)
                                    sanctioner.other_directed_emotion = 0

            
        if HOSPITAL in norm.get("antecedent").get("location"):
            agents = self.grid.get_cell_list_contents([(HOSPITAL,0)])
            agents = [agent for agent in agents if agent.health != DECEASED ]
            violate_agents = [agent for agent in agents if getattr(agent, norm.get("antecedent").get("attribute")) in norm.get("antecedent").get("value") ]
            
            if len(agents) - len(violate_agents) > 0:
                for agent in violate_agents:
                    sanctioners = [sanctioner for sanctioner in agents if sanctioner.health == NOT_INFECTED and sanctioner.unique_id != agent.unique_id]
                    
                    hard_sanction, sanction = False, False
                    p = self.random.uniform(0, 1)
                    if p < sanction_prob_w_health[agent.perceived_health]:
                        p = self.random.uniform(0, 1)
                        if p < hard_sanctioning_prob:
                            agent.FQ_frames = FQ_frames
                            hard_sanction = True
                            
                        elif agent_type == agent_type_message:
                            if p < message_sanctioning_prob:
                                sanction = True
                                agent.external_reward += norm.get("consequent").get("message")
                        elif agent_type in (agent_type_hint, agent_type_emote):
                            if p < emotion_sanctioning_prob:
                                sanction = True
                                if agent_type == agent_type_hint:
                                    agent.external_reward += norm.get("consequent").get("emotion")
                                agent.other_directed_emotion = -emotion_payoff
                
                    if agent_type in (agent_type_hint, agent_type_emote) and sanction:
                        for sanctioner in sanctioners:
                            if sanctioner.unique_id not in emotion_saction_set:
                                emotion_saction_set.add(sanctioner.unique_id)
                                sanctioner.other_directed_emotion = 0

    def perceive(self, x, y, agent_id):
        agents = self.grid.get_cell_list_contents([(x, y)])
        emotions = [agent.other_directed_emotion for agent in agents if agent.unique_id != agent_id]
        return sum(emotions)/len(emotions) if len(emotions) > 0 else 0
    
    def learn(self):
        for i in range(self.num_agents):
            agent = self.schedule._agents[i]
            agent.learn()
    
    def compute_infected(self):
        infectedAgents = [agent for agent in self.schedule.agents if agent.health in (INFECTED_A, INFECTED_S, CRITICAL) ]
        return len(infectedAgents)
    
    def compute_deceased(self):
        deceasedAgents = [agent for agent in self.schedule.agents if agent.health == DECEASED ]
        return len(deceasedAgents)
    
    def compute_compliance(self):
        agents = [agent for agent in self.schedule.agents if agent.last_action == MOVE_HOME and agent.health not in (NOT_INFECTED, DECEASED) ]
        return len(agents)
    
    def compute_violation(self):
        agents = [agent for agent in self.schedule.agents if agent.last_action != MOVE_HOME and agent.health not in (NOT_INFECTED, DECEASED) ]
        return len(agents)
    
    def compute_compliance_rate(self):
        comp = self.compute_compliance()
        vio = self.compute_violation()
        return comp / (comp + vio) if comp + vio > 0 else 0
    
    def compute_vaccinated(self):
        agents = [agent for agent in self.schedule.agents if agent.vaccinated and agent.health != DECEASED ]
        return len(agents)
    
    def compute_QC(self):
        count = 0
        for y in range(num_agents // agent_per_home):
            agents = self.grid.get_cell_list_contents([(HOME,y)])
            count += len(agents)
        return count
    
    def compute_FQ(self):
        count = 0
        for y in range(num_agents // agent_per_home):
            agents = self.grid.get_cell_list_contents([(HOME,y)])
            agents = [agent for agent in agents if agent.FQ_frames > 1]
            count += len(agents)
        return count
    
    def step(self):
        self.schedule.step()
        if agent_type != agent_type_primitive:
            self.run_norms()
        self.identifyAgentsAndUpdateSpread()
        self.learn()
        self.write_csv()
        self.datacollector.collect(self)
        self.decay_emotion()
    
    def decay_emotion(self):
        for i in range(self.num_agents):
            agent = self.schedule._agents[i]
            agent.other_directed_emotion = 0
            agent.self_directed_emotion = 0

    def write_csv(self):
        if self.iteration <= num_train_iterations:
            return
        
        with open(logs_path + file_name + '_agent.csv', 'a', newline = '') as agent_file:
            writer = csv.writer(agent_file, delimiter = ',')
            for i in range(self.num_agents):
                agent = self.schedule._agents[i]
                if agent_type in (agent_type_primitive, agent_type_sanctioning):
                    writer.writerow([self.iteration, self.schedule.steps, agent.unique_id, agent.health, agent.infected_times, agent.reward, agent.FQ_frames, agent.vaccinated, agent.vaccine_last, agent.intention, agent.last_action, agent.x, agent.y, agent.self_directed_emotion, agent.other_directed_emotion])
                elif agent_type == agent_type_message:
                    writer.writerow([self.iteration, self.schedule.steps, agent.unique_id, agent.health, agent.infected_times, agent.reward, agent.FQ_frames, agent.vaccinated, agent.vaccine_last, agent.intention, agent.last_action, agent.x, agent.y, agent.self_directed_emotion, agent.other_directed_emotion])
                elif agent_type == agent_type_emote:
                    writer.writerow([self.iteration, self.schedule.steps, agent.unique_id, agent.health, agent.infected_times, agent.reward, agent.FQ_frames, agent.vaccinated, agent.vaccine_last, agent.intention, agent.last_action, agent.x, agent.y, agent.self_directed_emotion, agent.other_directed_emotion])
                else:
                    writer.writerow([self.iteration, self.schedule.steps, agent.unique_id, agent.health, agent.infected_times, agent.reward, agent.FQ_frames, agent.vaccinated, agent.vaccine_last, agent.intention, agent.last_action, agent.x, agent.y, agent.self_directed_emotion, agent.other_directed_emotion])
        agent_file.close()
    
    def identifyAgentsAndUpdateSpread(self):
        # Park
        for y in range(num_parks):
            agents = self.grid.get_cell_list_contents([(PARK,y)])
            self.updateSpread(agents)
        # Grocery
        for y in range(num_grocery_stores):
            agents = self.grid.get_cell_list_contents([(GROCERY_STORE,y)])
            self.updateSpread(agents)
        # Home
        for y in range(num_agents // agent_per_home):
            agents = self.grid.get_cell_list_contents([(HOME,y)])
            self.updateSpread(agents, True)
    def updateSpread(self, agents, close_contact = False):
        if any(a.health in (INFECTED_A, INFECTED_S, CRITICAL) for a in agents):
            [a.infect(close_contact) for a in agents]

In [54]:
def run_simulation(iteration, evaluate):
    model = SimulationModel(num_agents, iteration, evaluate)
    for i in range(num_steps):
        model.step()
    modelDF = model.datacollector.get_model_vars_dataframe()
    return model.deceasedCount

In [ ]:
infected, deceased, quarantine, compliance, violation, ratio, vaccinated = None, None, None, None, None, None, None
for i in tqdm(range(1, num_train_iterations + num_eval_iterations + 1)):
    deceasedCount = run_simulation(i, True if i > num_train_iterations else False)
    print('deceasedCount', deceasedCount)

  1%|▉                                                                                | 1/90 [00:52<1:17:10, 52.03s/it]

deceasedCount 79


  2%|█▊                                                                               | 2/90 [01:38<1:11:15, 48.58s/it]

deceasedCount 79


  3%|██▋                                                                              | 3/90 [02:24<1:08:47, 47.44s/it]

...

deceasedCount 13


 96%|████████████████████████████████████████████████████████████████████████████▍   | 86/90 [2:02:00<05:30, 82.58s/it]

deceasedCount 6


 97%|█████████████████████████████████████████████████████████████████████████████▎  | 87/90 [2:03:17<04:02, 80.98s/it]

deceasedCount 16
